In [5]:
[ ]
# Part 0: Data Preparation
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



adult = fetch_openml(name="adult", version=2, as_frame=True)
df = adult.frame


X = df.drop(columns="class")
y = df["class"]


X_train_full, X_temp, y_train_full, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


categorical = X.select_dtypes(include=["category", "object"]).columns
numerical = X.select_dtypes(include=["int64", "float64"]).columns


preprocessor = ColumnTransformer([
("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
("num", StandardScaler(), numerical)
])


X_train = preprocessor.fit_transform(X_train_full)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train_full)
y_val = le.transform(y_val)
y_test = le.transform(y_test)


X_train.shape, X_val.shape, X_test.shape


[ ]

[]

In [6]:
[ ]
# Part 1: Optimizers
import tensorflow as tf
from tensorflow import keras

def build_model():
    model = keras.Sequential([
        keras.layers.Input(shape=(X_train.shape[1],)),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid")
    ])
    return model

optimizers = {
    "SGD": keras.optimizers.SGD(learning_rate=0.01),
    "SGD_momentum": keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    "Adam": keras.optimizers.Adam(learning_rate=0.001)
}

histories = {}
for name, opt in optimizers.items():
    model = build_model()
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=0)
    histories[name] = history

for name, history in histories.items():
    print(name, "Train Acc:", history.history['accuracy'][-1], "Val Acc:", history.history['val_accuracy'][-1])


SGD Train Acc: 0.857498049736023 Val Acc: 0.861725389957428
SGD_momentum Train Acc: 0.8595747351646423 Val Acc: 0.8592683672904968
Adam Train Acc: 0.8674427270889282 Val Acc: 0.8640458583831787


In [7]:
[ ]
# Part 2: Batch Size
batch_sizes = [1, 32, 128, 1024]

for bs in batch_sizes:
    model = build_model()
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=bs, epochs=5, verbose=0)
    print(f"Batch {bs}: Train Acc {history.history['accuracy'][-1]:.4f}, Val Acc {history.history['val_accuracy'][-1]:.4f}")


Batch 1: Train Acc 0.8600, Val Acc 0.8634
Batch 32: Train Acc 0.8597, Val Acc 0.8617
Batch 128: Train Acc 0.8582, Val Acc 0.8593
Batch 1024: Train Acc 0.8553, Val Acc 0.8608


In [8]:
[ ]
# Part 3: Overfitting and Regularization
# Large model (overfitting expected)
model_large = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model_large.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history_large = model_large.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, verbose=0)

# L2 regularization
model_l2 = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(1, activation="sigmoid")
])
model_l2.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history_l2 = model_l2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, verbose=0)

# Dropout regularization
model_dropout = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])
model_dropout.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history_dropout = model_dropout.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, verbose=0)

print("Large model Val Acc:", history_large.history['val_accuracy'][-1])
print("L2 Reg Val Acc:", history_l2.history['val_accuracy'][-1])
print("Dropout Val Acc:", history_dropout.history['val_accuracy'][-1])


Large model Val Acc: 0.845754861831665
L2 Reg Val Acc: 0.8615888357162476
Dropout Val Acc: 0.8624078631401062


In [9]:
[ ]
# Part 4: Early Stopping
model_es = build_model()
model_es.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
history_es = model_es.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, callbacks=[es], verbose=0)

print("Trained epochs:", len(history_es.history['accuracy']))
print("Best Val Acc:", max(history_es.history['val_accuracy']))


Trained epochs: 9
Best Val Acc: 0.8641823530197144
